В этом ноутбуке будут доработки структуры данных, описание которой было начато в ноутбуке osm_data_structure.

Планируется, что основные доработки будут состоять в следующем:

- в круговой диаграмме в пределах одного сектора будет хранится отсортированный по расстоянию список объектов
- возможно, также будет для каждого сектора хранится в том или ином виде список расстояний до объектов


In [371]:
from openstreetmap_load_data import find_objects_bounds
from utils import calculate_right_top_coordinates
from google_api_load_data import load_prepare_and_cut_image
from osm_object import One_OSM_object, Image_OSM_object

In [372]:
import numpy as np
import math

**Итак, поехали, для начала загрузим информацию об одной структуре osm и сформируем из неё Image_OSM_object**

In [373]:
left_bottom_lat = 54.4081
left_bottom_lon = 48.980
right_top_lat, right_top_lon = calculate_right_top_coordinates(left_bottom_lat, left_bottom_lon)


datafile_name = "tmp.json"

bounds, tags = find_objects_bounds(datafile_name, right_top_lat, right_top_lon, left_bottom_lat, left_bottom_lon)

In [374]:
print(bounds[0])
print(tags[0])

{'maxlat': 54.647034467866604, 'minlat': 54.6400339, 'maxlon': 48.985209, 'minlon': 48.98}
water


In [416]:
# img_cutted_satellite = load_prepare_and_cut_image(left_bottom_lat=left_bottom_lat, left_bottom_lon=left_bottom_lon,
#                                                   right_top_lat=right_top_lat, right_top_lon=right_top_lon,
#                                                   is_debug=True)

In [376]:
# Ну и наконец создаём нужный объект

In [377]:
img_osm_1 = Image_OSM_object(img_cutted_satellite.shape, right_top_lat, right_top_lon,
                             left_bottom_lat, left_bottom_lon, bounds, tags)

In [378]:
img_osm_1.osm_objects[8].print_osm_object()

maxlat: 0, minlat: 250, maxlon: 399, minlon: 104


In [379]:
len(img_osm_1.osm_objects)

44

**Теперь из osm_data_structure.ipynb перенесём сюда всякие базовые функции для подсчёта углов и вот это вот всё**

In [380]:
def calculate_angle(vector_1, vector_2):
    scalar_product = np.dot(vector_1, vector_2)
    norm_1 = np.linalg.norm(vector_1)
    norm_2 = np.linalg.norm(vector_2) 
    return np.rad2deg(np.arccos(scalar_product / (norm_1 * norm_2)))


def calculate_directed_angle(vector_1, vector_2):
    
    angle = calculate_angle(vector_1, vector_2)
    
    pseudoscalar_product = vector_1[0] * vector_2[1] - vector_1[1] * vector_2[0]
    norm_1 = np.linalg.norm(vector_1)
    norm_2 = np.linalg.norm(vector_2) 
    dir_angle = np.rad2deg(np.arcsin(pseudoscalar_product / (norm_1 * norm_2)))
    
    if dir_angle < 0:
        angle = 360 - angle
    return angle    


def calculate_border_angles_to_object_and_point(x, y, osm_object):
    """
    Функция, вычисляющая для одного osm-объекта два угла, между которыми можно видеть объект, находясь в точке (x, y)
    """
    minlat = osm_object.minlat
    maxlat = osm_object.maxlat
    minlon = osm_object.minlon
    maxlon = osm_object.maxlon

    # Получаем 4 вектора для каждой угловой точки границы объекта
    v1 = (maxlat - x, minlon - y)
    v2 = (maxlat - x, maxlon - y)
    v3 = (minlat - x, minlon - y)
    v4 = (minlat - x, maxlon - y)
    
    # Для каждой пары векторов посчитаем угол между ними
    vectors = [v1, v2, v3, v4]
#     print(vectors)
    angle_vector_pair = {}
    for i in range(4):
        for j in range(i, 4):
            if i != j:
                vector_1 = vectors[i]
                vector_2 = vectors[j]

                angle = abs(calculate_angle(vector_1, vector_2))
                angle_vector_pair[angle] = (i, j)
                
    # Теперь нужно определить, какие 2 вектора дают максимальный угол
    max_angle = max(angle_vector_pair)
    vector_pair = angle_vector_pair[max_angle]
    vector_1 = vectors[vector_pair[0]]
    vector_2 = vectors[vector_pair[1]]

    # Определяем угол между каждым из векторов и осью абсцисс, начинающейся в точке (x, y)
    abscissa = (0, img_cutted_satellite.shape[1] - y)
    
    angle_1 = calculate_directed_angle(abscissa, vector_1)
    angle_2 = calculate_directed_angle(abscissa, vector_2)
    
    return angle_1, angle_2 

На данном этапе мы верим, что функции, представленные выше работают корректно (они же были уже проверены в osm_data_structure.ipynb (Да? Были же?))

**Теперь добавим учёт расстояния**

In [381]:
def calculate_distance(point_1, point_2):
    """
    Определяет евклидово расстояние между двумя точками
    """
    x1, y1 = point_1
    x2, y2 = point_2

    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

def calculate_osm_object_center(osm_object):
    minlat = osm_object.minlat
    maxlat = osm_object.maxlat
    minlon = osm_object.minlon
    maxlon = osm_object.maxlon
    
    return (minlat - maxlat, maxlon - minlon)

In [382]:
img_osm_1.osm_objects[8].print_osm_object()

maxlat: 0, minlat: 250, maxlon: 399, minlon: 104


In [383]:
calculate_osm_object_center(img_osm_1.osm_objects[8])

(250, 295)

In [384]:
class Circle_diagram(object):
    def __init__(self, sectors_count, center_point=(0., 0.)):
        self.sectors_count = sectors_count
        self.center_point = center_point
    
        # Количество градусов, которое помещается в каждом секторе
        self.step_grad = 360 / sectors_count
        
        self.sectors = [[] for _ in range(sectors_count)]
        
    def insert(self, osm_object):
        center_x = self.center_point[0]
        center_y = self.center_point[1]
        angle_1, angle_2 = calculate_border_angles_to_object_and_point(center_x, center_y, osm_object)
        object_x, object_y = calculate_osm_object_center(osm_object)
        
        distance = calculate_distance((center_x, center_y), (object_x, object_y))
        
        diff_between_angles = abs(angle_2 - angle_1)
        if diff_between_angles < 180: # Перехода через 0 нет
            start_angle = min(angle_1, angle_2)
            end_angle = max(angle_1, angle_2)
            
        if diff_between_angles >= 180: # Есть переход через 0
            start_angle = max(angle_1, angle_2)
            end_angle = min(angle_1, angle_2)
            
        sector_start_idx = int(start_angle / self.step_grad)
        sector_end_idx = int(end_angle / self.step_grad)
        
        if diff_between_angles < 180:
            for sector_idx in range(sector_start_idx, sector_end_idx + 1):    
                self.sectors[sector_idx].append((distance, osm_object.tag))
        else:
            for sector_idx in list(range(sector_start_idx, self.sectors_count)) + list(range(0, sector_end_idx)):
                self.sectors[sector_idx].append((distance, osm_object.tag))
        
    def resort_sectors(self):
        """
        Упорядочевает объекты внутри сектора по степени увеличения расстояния до объекта
        """
        for i in range(len(self.sectors)):
            self.sectors[i] = sorted(self.sectors[i])
            
    def rotate(self):
        """
        'Поворачивает' диаграмму на 1 сектор
        """
        self.sectors = [self.sectors[-1]] + self.sectors[:-1]
            

In [385]:
# for i in range(len(img_osm_1.osm_objects)):
#     print(i)
#     img_osm_1.osm_objects[i].print_osm_object()

In [386]:
cd = Circle_diagram(8, (200, 200))

for i in range(len(img_osm_1.osm_objects)):
    osm_object = img_osm_1.osm_objects[i]
    cd.insert(osm_object)
    
cd.resort_sectors()
    
for i in range(8):
    print(cd.sectors[i])
    print()

[(270.1740179958095, 'wood')]

[(107.35455276791944, 'wood')]

[(107.35455276791944, 'wood'), (189.02909828912584, 'wood'), (246.272207120495, 'wood'), (249.897979183506, 'water'), (253.53895164254348, 'wood'), (268.01679051880313, 'wood'), (269.4160351575236, 'wood'), (270.1147904132612, 'wood'), (270.8671260969112, 'wood'), (274.36107595648474, 'water'), (274.36107595648474, 'wood')]

[(107.35455276791944, 'wood'), (189.02909828912584, 'wood'), (237.26145915424192, 'wood'), (253.53895164254348, 'wood'), (260.10959228755866, 'wood'), (260.27677575995904, 'wood'), (261.0383113644432, 'water'), (263.1387466717891, 'wood'), (264.59402865522117, 'wood'), (270.1147904132612, 'wood'), (270.8671260969112, 'wood'), (271.5290039756342, 'wood'), (272.237029075767, 'water'), (272.2443755158222, 'wood'), (272.94321753800733, 'wood'), (273.6512378923216, 'wood'), (274.36107595648474, 'wood'), (274.37201023428025, 'wood')]

[(107.35455276791944, 'wood'), (229.10696191953662, 'wood'), (237.261459154

In [387]:
sectors_count = 100

cd = Circle_diagram(sectors_count, (200, 200))

for i in range(len(img_osm_1.osm_objects)):
    osm_object = img_osm_1.osm_objects[i]
    cd.insert(osm_object)
    
cd.resort_sectors()
    
for i in range(sectors_count):
    print(cd.sectors[i])
    print()

[]

[]

[]

[]

[]

[]

[]

[(270.1740179958095, 'wood')]

[(270.1740179958095, 'wood')]

[(270.1740179958095, 'wood')]

[(270.1740179958095, 'wood')]

[]

[(107.35455276791944, 'wood')]

[(107.35455276791944, 'wood')]

[(107.35455276791944, 'wood')]

[(107.35455276791944, 'wood')]

[(107.35455276791944, 'wood')]

[(107.35455276791944, 'wood')]

[(107.35455276791944, 'wood')]

[(107.35455276791944, 'wood')]

[(107.35455276791944, 'wood')]

[(107.35455276791944, 'wood')]

[(107.35455276791944, 'wood')]

[(107.35455276791944, 'wood')]

[(107.35455276791944, 'wood')]

[(107.35455276791944, 'wood')]

[(107.35455276791944, 'wood')]

[(107.35455276791944, 'wood')]

[(107.35455276791944, 'wood')]

[(107.35455276791944, 'wood')]

[(107.35455276791944, 'wood')]

[(107.35455276791944, 'wood'), (189.02909828912584, 'wood')]

[(107.35455276791944, 'wood'), (189.02909828912584, 'wood'), (249.897979183506, 'water')]

[(107.35455276791944, 'wood'), (189.02909828912584, 'wood'), (246.272207120495, 'wo

In [388]:
sectors_count = 100

cd = Circle_diagram(sectors_count, (5, 5))

for i in range(len(img_osm_1.osm_objects)):
    osm_object = img_osm_1.osm_objects[i]
    cd.insert(osm_object)
    
cd.resort_sectors()
    
for i in range(sectors_count):
    print(cd.sectors[i])
    print()

[(105.65036677645753, 'wood')]

[(105.65036677645753, 'wood')]

[(105.65036677645753, 'wood')]

[(105.65036677645753, 'wood')]

[(105.65036677645753, 'wood')]

[(105.65036677645753, 'wood')]

[(105.65036677645753, 'wood')]

[(105.65036677645753, 'wood')]

[(105.65036677645753, 'wood')]

[(105.65036677645753, 'wood')]

[(105.65036677645753, 'wood')]

[(105.65036677645753, 'wood')]

[(105.65036677645753, 'wood')]

[(105.65036677645753, 'wood')]

[(105.65036677645753, 'wood')]

[(105.65036677645753, 'wood')]

[(105.65036677645753, 'wood')]

[(105.65036677645753, 'wood')]

[(105.65036677645753, 'wood')]

[(105.65036677645753, 'wood')]

[(105.65036677645753, 'wood')]

[(105.65036677645753, 'wood')]

[(105.65036677645753, 'wood')]

[(105.65036677645753, 'wood')]

[(105.65036677645753, 'wood')]

[(105.65036677645753, 'wood')]

[(105.65036677645753, 'wood')]

[(105.65036677645753, 'wood')]

[(105.65036677645753, 'wood')]

[(105.65036677645753, 'wood')]

[(105.65036677645753, 'wood')]

[(105.65

In [389]:
sectors_count = 100

cd = Circle_diagram(sectors_count, (125, 45))

for i in range(len(img_osm_1.osm_objects)):
    osm_object = img_osm_1.osm_objects[i]
    cd.insert(osm_object)
    
cd.resort_sectors()
    
for i in range(sectors_count):
    print(cd.sectors[i])
    print()

[(279.5084971874737, 'wood')]

[(279.5084971874737, 'wood')]

[(279.5084971874737, 'wood')]

[(279.5084971874737, 'wood')]

[(279.5084971874737, 'wood')]

[(279.5084971874737, 'wood')]

[(119.85407794480753, 'wood'), (279.5084971874737, 'wood')]

[(119.85407794480753, 'wood'), (122.05326705991938, 'wood'), (279.5084971874737, 'wood')]

[(119.85407794480753, 'wood'), (122.05326705991938, 'wood'), (279.5084971874737, 'wood')]

[(119.85407794480753, 'wood'), (122.05326705991938, 'wood'), (279.5084971874737, 'wood')]

[(121.45781160551181, 'wood'), (122.05326705991938, 'wood'), (279.5084971874737, 'wood')]

[(121.45781160551181, 'wood'), (122.05326705991938, 'wood'), (279.5084971874737, 'wood')]

[(121.45781160551181, 'wood'), (122.05326705991938, 'wood'), (279.5084971874737, 'wood')]

[(110.76551810017412, 'water'), (117.95338062132852, 'wood'), (121.45781160551181, 'wood'), (122.05326705991938, 'wood'), (279.5084971874737, 'wood')]

[(106.6770828247567, 'wood'), (110.76551810017412, 'wat

Теперь нужно написать функцию для создания множества диаграмм для конкретного изображения, а также функцию для создания диаграммы агента (без указания расстояний до объектов, но с упорядочиванием по удалённости)

In [390]:
def create_circle_diagram(sectors_count, center_point, img_osm):
    cd = Circle_diagram(sectors_count, center_point)
    
    for i in range(len(img_osm.osm_objects)):
        osm_object = img_osm.osm_objects[i]
        cd.insert(osm_object)
    
    cd.resort_sectors()
    return cd

In [391]:
cd = create_circle_diagram(sectors_count = 30, center_point=(100, 100), img_osm=img_osm_1)

for i in range(len(cd.sectors)):
    print(cd.sectors[i])
    print()

[(246.01829200285087, 'wood')]

[(246.01829200285087, 'wood')]

[(128.00390619039717, 'wood'), (246.01829200285087, 'wood')]

[(128.00390619039717, 'wood'), (246.01829200285087, 'wood')]

[(128.00390619039717, 'wood'), (246.01829200285087, 'wood')]

[(128.00390619039717, 'wood'), (246.01829200285087, 'wood')]

[(108.85311203635843, 'water'), (126.62148316932637, 'wood'), (128.00390619039717, 'wood'), (132.94359706281458, 'water'), (246.01829200285087, 'wood')]

[(64.28063471995279, 'wood'), (105.11898020814318, 'wood'), (108.85311203635843, 'water'), (126.62148316932637, 'wood'), (132.94359706281458, 'water')]

[(64.28063471995279, 'wood'), (105.11898020814318, 'wood'), (108.85311203635843, 'water')]

[(64.28063471995279, 'wood'), (105.11898020814318, 'wood')]

[(64.28063471995279, 'wood'), (105.11898020814318, 'wood'), (112.61438629233834, 'wood'), (128.69343417595164, 'wood'), (129.49517365523704, 'wood'), (132.94359706281458, 'wood')]

[(64.28063471995279, 'wood'), (112.614386292338

In [392]:
# Для структуры хранящей все центры сетки, а также множество круговых диаграмм для изображения, кажется,
# подойдёт соответствующий класс

In [393]:
img_cutted_satellite.shape

(300, 400, 3)

In [394]:
# Здесь по количеству элементов сетки (с большей стороны изображения) нужно получить количество секторов,
# на которые нужно разбить диаграмму

In [395]:
def calculate_angle_by_blocks_count(t):
    """
    t - количество блоков на большей стороне изображения
    (см формулы, там это обозначается именно как t)
    
    Возвращает угол сектора круговой диаграммы В ГРАДУСАХ
    """
    numerator = 2 * t_sq_d / 4 + t_sq_d * (t - 3 / 2) ** 2 - t_sq_d + (1 - 1 / (2 * t)) ** 2
    denominator = 1 / t * math.sqrt(1 + 4 * (t - 3 / 2) ** 2) * math.sqrt(t_sq_d / 4 + (1 - 1 / (2 * t)) ** 2)
    return math.degrees(math.acos(numerator / denominator))


def calculate_blocks_centers(img_osm_shape, bigger_size_blocks_count):
    """
    Определяет координаты центров элементов сетки на изображении в пикселях
    """
    height, width, _ = img_osm_shape
    
    bigger_side_len = height
    if width > bigger_side_len:
        bigger_side_len = width
    
    # Большая сторона должна делиться на количество элементов нацело
    one_element_len = bigger_side_len / bigger_size_blocks_count
    assert(one_element_len % 1 == 0)
    one_element_len = int(one_element_len)
    
    blocks_count_height = int(height / one_element_len)
    blocks_count_width = int(width / one_element_len)
    
    height_dots = []
    width_dots = []
    
    for i in range(blocks_count_height):
        height_dots.append(i * one_element_len + one_element_len / 2)
    for i in range(blocks_count_width):
        width_dots.append(i * one_element_len + one_element_len / 2)
    
    center_points = []
    for i in range(blocks_count_height):
        for j in range(blocks_count_width):
            center_points.append((height_dots[i], width_dots[j]))
            
    return center_points


class Image_circle_diagrams(object):
    def __init__(self, img_osm, bigger_size_blocks_count):
        """
        img_osm - структура объектов на изображении (см utils)
        bigger_size_blocks_count - количество элементов сетки на большей стороне изображения
        """
        self.sector_angle = calculate_angle_by_blocks_count(bigger_size_blocks_count)
        # Округляем вверх, чтобы наверняка гарантировать точность
        self.sectors_count = math.ceil(360. / self.sector_angle)
        print(self.sectors_count)
        
        # Дальше нужно понять, сколько будет блоков всего на изображении, и где будут находиться центры этих блоков (сетки)
        self.center_points = calculate_blocks_centers(img_osm.img_cutted_shape, bigger_size_blocks_count)
        
        # Теперь для каждой из центральных точек нужно построить круговую диаграмму
        self.circle_diagrams = []
        for center_p in self.center_points:
            self.circle_diagrams.append(create_circle_diagram(self.sectors_count, center_p, img_osm))
        

In [396]:
calculate_angle_by_blocks_count(4)

3.1798301198641643

In [397]:
calculate_blocks_centers(img_osm_1.img_cutted_shape, 4)

[(50.0, 50.0),
 (50.0, 150.0),
 (50.0, 250.0),
 (50.0, 350.0),
 (150.0, 50.0),
 (150.0, 150.0),
 (150.0, 250.0),
 (150.0, 350.0),
 (250.0, 50.0),
 (250.0, 150.0),
 (250.0, 250.0),
 (250.0, 350.0)]

In [398]:
img_cd = Image_circle_diagrams(img_osm_1, 4)

114


C:\Users\vruch\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in arccos
  """


In [399]:
img_osm_1.img_cutted_shape

(300, 400, 3)

In [400]:
for i in range(len(img_cd.circle_diagrams)):
    print(img_cd.center_points[i])
    for sector in img_cd.circle_diagrams[i].sectors:
        print(sector)
    print()

(50.0, 50.0)
[(35.35533905932738, 'wood'), (43.382023926967726, 'wood'), (55.97320787662612, 'wood'), (58.89821729050889, 'wood'), (316.26729201736936, 'wood')]
[(35.35533905932738, 'wood'), (43.382023926967726, 'wood'), (55.97320787662612, 'wood'), (58.89821729050889, 'wood'), (316.26729201736936, 'wood')]
[(35.35533905932738, 'wood'), (39.35733730830886, 'water'), (43.382023926967726, 'wood'), (55.97320787662612, 'wood'), (58.89821729050889, 'wood'), (62.24146527838174, 'water'), (316.26729201736936, 'wood')]
[(35.35533905932738, 'wood'), (39.35733730830886, 'water'), (43.382023926967726, 'wood'), (55.97320787662612, 'wood'), (58.89821729050889, 'wood'), (62.24146527838174, 'water'), (316.26729201736936, 'wood')]
[(35.35533905932738, 'wood'), (39.35733730830886, 'water'), (43.382023926967726, 'wood'), (57.723478758647246, 'wood'), (58.89821729050889, 'wood'), (62.24146527838174, 'water'), (316.26729201736936, 'wood')]
[(35.35533905932738, 'wood'), (39.35733730830886, 'water'), (43.38

**Идём дальше: КРУГОВАЯ ДИАГРАММА АГЕНТА**

In [401]:
class Agent_circle_diagram(Circle_diagram):
    def __init__(self, sectors_count):
        self.sectors_count = sectors_count
    
        # Количество градусов, которое помещается в каждом секторе
        self.step_grad = 360 / sectors_count
        
        self.sectors = [[] for _ in range(sectors_count)]
    
    def insert(self, angle_1, angle_2, closeness_priority, tag):
        """
        Заметим следующее:
        объект может находиться, разумеется в нескольких секторах, в разных секторах у него может быть разный приоритет,
        например, у леса в одном секторе приоритет 1, а в другом 2
        В таком случае в круговую диаграмму добавляется несколько объектов, соответствующих одному на изображении:
        в приведённом выше примере будет добавлено 2 объекта - лес в одном секторе с приоритетом 1 и в другом с приоритетом 2
        
        angle_1 и angle_2 - значения в градусах
        closeness_priority - целое число от 1 до ..., чем дальше объект, тем выше closeness_priority
        tag - строка, тип объекта
        """
        diff_between_angles = abs(angle_2 - angle_1)
        if diff_between_angles < 180: # Перехода через 0 нет
            start_angle = min(angle_1, angle_2)
            end_angle = max(angle_1, angle_2)
            
        if diff_between_angles >= 180: # Есть переход через 0
            start_angle = max(angle_1, angle_2)
            end_angle = min(angle_1, angle_2)
            
        sector_start_idx = int(start_angle / self.step_grad)
        sector_end_idx = int(end_angle / self.step_grad)
        
        if diff_between_angles < 180:
            for sector_idx in range(sector_start_idx, sector_end_idx + 1):    
                self.sectors[sector_idx].append((closeness_priority, tag))
        else:
            for sector_idx in list(range(sector_start_idx, self.sectors_count)) + list(range(0, sector_end_idx)):
                self.sectors[sector_idx].append((closeness_priority, tag))
        

In [402]:
agent_cd = Agent_circle_diagram(114)
agent_cd.insert(50, 60, 2, 'water')
agent_cd.insert(30, 60, 1, 'wood')
agent_cd.insert(100, 120, 1, 'water')
agent_cd.insert(65, 90, 1, 'water')

agent_cd.resort_sectors()

In [403]:
agent_cd.sectors

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [(1, 'wood')],
 [(1, 'wood')],
 [(1, 'wood')],
 [(1, 'wood')],
 [(1, 'wood')],
 [(1, 'wood')],
 [(1, 'wood'), (2, 'water')],
 [(1, 'wood'), (2, 'water')],
 [(1, 'wood'), (2, 'water')],
 [(1, 'wood'), (2, 'water')],
 [(1, 'wood'), (2, 'water')],
 [(1, 'water')],
 [(1, 'water')],
 [(1, 'water')],
 [(1, 'water')],
 [(1, 'water')],
 [(1, 'water')],
 [(1, 'water')],
 [(1, 'water')],
 [(1, 'water')],
 [],
 [],
 [(1, 'water')],
 [(1, 'water')],
 [(1, 'water')],
 [(1, 'water')],
 [(1, 'water')],
 [(1, 'water')],
 [(1, 'water')],
 [(1, 'water')],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 []]

**ЛОГИКА ШТРАФОВ**

$penalty_s = \sum_{i = 1}^{\max(k_1, k_2)}\left(\frac{1}{i^2} \cdot I (F_i \ne S_i) \right)$

где $k_1$ и $k_2$ - количества наименований объектов, содержащихся в первом и втором секторе соответственно, $F_i$ - наименование $i$-го по порядку объекта из первого сектора, $S_i$ - наименование $i$-го по порядку объекта из второго сектора, индекс $i$ отвечает за номер рассматриваемой пары объектов (для объекта, для которого нет соответствующей пары по номеру из другой диаграммы считается, что этот объект не равен объекту, который должен ему соответствовать, то есть индикатор в формуле равен единице), а $s$ - номер рассматриваемой пары секторов.

In [404]:
def calculate_penalty_for_sectors(sector_img_diagram, sector_agent_diagram):
    """
    Принимает по одному сектору из каждой диаграммы
    sector_img_diagram - сектор одной из диаграмм для изображения
    sector_agent_diagram - сектор диаграммы агента
    Секторы должны соответствовать друг другу по порядку
    
    Каждый из секторов представляет собой список пар: (расстояние или приоритет, тип объекта)
    Считаем, что объекты внутри сектора уже отсортированы в порядке увеличения расстояния до них
    """
    
    min_length = min(len(sector_img_diagram), len(sector_agent_diagram))
    
    penalty = 0
    
    for i in range(min_length):
        indicator = 0
        if sector_img_diagram[i][1] != sector_agent_diagram[i][1]:
            indicator = 1
        penalty += (indicator / (i + 1) ** 2)
    
    more_sector_len = max(len(sector_img_diagram[min_length:]), len(sector_agent_diagram[min_length:]))
    for i in range(min_length, min_length + more_sector_len):
        penalty += (1 / (i + 1) ** 2)
        
    return penalty

In [405]:
calculate_penalty_for_sectors(
    [(0.5, 'wood'), (1.2, 'water'), (1.5, 'wood')],
    [(1, 'water'), (2, 'wood')]
)

1.3611111111111112

***Тесты***

In [406]:
print("Test 1")
assert(
    calculate_penalty_for_sectors(
        [(100.1, 'water'), (134, 'wood')],
        [(1, 'water'), (2, 'wood')]
    ) == 0)
print("OK")
print("------------------------------------------")

print("Test 2")
assert(
    calculate_penalty_for_sectors(
        [(100, 'water')],
        []
    ) == 1
)
print("OK")
print("------------------------------------------")

print("Test 3")
assert(
    calculate_penalty_for_sectors(
        [(100, 'water'), (120, 'wood')],
        [(1, 'water')]
    ) == 0.25
)
print("OK")
print("------------------------------------------")

print("Test 4")
assert(
    calculate_penalty_for_sectors(
        [(100, 'water'), (120, 'wood')],
        [(1, 'wood'), (2, 'water')]
    ) == 1.25
)
print("OK")
print("------------------------------------------")

print("Test 5")
assert(
    calculate_penalty_for_sectors(
        [(100, 'water'), (120, 'wood'), (135, 'water')],
        [(1, 'wood'), (2, 'wood'), (3, 'wood')]
    ) == (1 + (1 / 9))
)
print("OK")
print("------------------------------------------")

print("Test 6")
assert(
    calculate_penalty_for_sectors(
        [(0.5, 'water'), (10, 'water'), (15, 'wood')],
        [(1, 'water'), (2, 'wood')]
    ) == (1 / 4 + 1 / 9)
)
print("OK")
print("------------------------------------------")

print("Test 7")
assert(
    calculate_penalty_for_sectors(
        [(0.5, 'water'), (12, 'wood'), (148, 'water')],
        [(1, 'water'), (2, 'wood')]
    ) == (1 / 9)
)
print("OK")
print("------------------------------------------")

Test 1
OK
------------------------------------------
Test 2
OK
------------------------------------------
Test 3
OK
------------------------------------------
Test 4
OK
------------------------------------------
Test 5
OK
------------------------------------------
Test 6
OK
------------------------------------------
Test 7
OK
------------------------------------------


Итоговый штраф для пары диаграмм вычисляем как сумму штрафов всех последовательных пар секторов.

In [407]:
def calculate_penalty_for_diagrams(img_sectors, agent_sectors):
    """
    Итоговый штраф для пары диаграмм вычислается как сумма штрафов всех последовательных пар секторов
    Принимает список секторов одной из круговых диаграмм для изображения и круговой диаграммы агента
    
    Возвращает число с плавающей точкой - суммарный штраф для пары диаграмм
    """
    assert(len(img_sectors) == len(agent_sectors))
    
    penalty_sum = 0
    
    for i in range(len(img_sectors)):
        penalty_sum += calculate_penalty_for_sectors(img_sectors[i], agent_sectors[i])
        
    return penalty_sum

In [408]:
calculate_penalty_for_diagrams(
    [
        [(199, 'wood'), (289, 'water')],
        [(199, 'wood')],
        [(389.7, 'water')]
    ],
    [
        [(1, 'wood')],
        [(1, 'wood'), (2, 'water')],
        [(1, 'water')]
    ]
)

0.5

***Тесты***

In [409]:
print("Test 1")
assert(
    calculate_penalty_for_diagrams(
        [
            [], [], [(100, 'water')]
        ],
        [
            [], [], [(1, 'water')]
        ]
    ) == 0
)
print("OK")
print("------------------------------------------")

print("Test 2")
assert(
    calculate_penalty_for_diagrams(
        [
            [(12, 'water'), (126, 'wood')], [(12, 'water')], [(126, 'wood')]
        ],
        [
            [(1, 'water'), (2, 'wood')], [(1, 'water')], [(1, 'wood')]
        ]
    ) == 0
)
print("OK")
print("------------------------------------------")

print("Test 3")
assert(
    calculate_penalty_for_diagrams(
        [
            [(12, 'water'), (126, 'wood')], [(12, 'water')], [(126, 'wood')]
        ],
        [
            [(1, 'water'), (2, 'water')], [(1, 'water')], [(1, 'water'), (2, 'wood')]
        ]
    ) == (1 / 4 + 1 + 1 / 4)
)
print("OK")
print("------------------------------------------")

print("Test 4")
assert(
    calculate_penalty_for_diagrams(
        [
            [(12, 'water'), (126, 'wood')], [(12, 'water')], [(126, 'wood')]
        ],
        [
            [(1, 'water')], [(1, 'wood')], [(1, 'water'), (2, 'wood')]
        ]
    ) == (1 / 4 + 1 + 1 + 1 / 4)
)
print("OK")
print("------------------------------------------")

print("Test 5")
assert(
    calculate_penalty_for_diagrams(
        [
            [(125, 'water')], [(125, 'water')], [(125, 'water')]
        ],
        [
            [], [], []
        ]
    ) == 3
)
print("OK")
print("------------------------------------------")

Test 1
OK
------------------------------------------
Test 2
OK
------------------------------------------
Test 3
OK
------------------------------------------
Test 4
OK
------------------------------------------
Test 5
OK
------------------------------------------


In [410]:
# Поворот круговой диаграммы

In [411]:
print(agent_cd.sectors)
print(len(agent_cd.sectors))
agent_cd.rotate()
print()
print(agent_cd.sectors)
print(len(agent_cd.sectors))

[[], [], [], [], [], [], [], [], [], [(1, 'wood')], [(1, 'wood')], [(1, 'wood')], [(1, 'wood')], [(1, 'wood')], [(1, 'wood')], [(1, 'wood'), (2, 'water')], [(1, 'wood'), (2, 'water')], [(1, 'wood'), (2, 'water')], [(1, 'wood'), (2, 'water')], [(1, 'wood'), (2, 'water')], [(1, 'water')], [(1, 'water')], [(1, 'water')], [(1, 'water')], [(1, 'water')], [(1, 'water')], [(1, 'water')], [(1, 'water')], [(1, 'water')], [], [], [(1, 'water')], [(1, 'water')], [(1, 'water')], [(1, 'water')], [(1, 'water')], [(1, 'water')], [(1, 'water')], [(1, 'water')], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
114

[[], [], [], [], [], [], [], [], [], [], [(1, 'wood')], [(1, 'wood')], [(1, 'wood')], [(1, 'wood')], [(1, 'wood')], [(1, 'wood')], [(1, 'wood')

**Выбор наиболее вероятного элемента сетки по минимальному штрафу**

In [412]:
img_cd.circle_diagrams

In [413]:
img_cd.center_points

[(50.0, 50.0),
 (50.0, 150.0),
 (50.0, 250.0),
 (50.0, 350.0),
 (150.0, 50.0),
 (150.0, 150.0),
 (150.0, 250.0),
 (150.0, 350.0),
 (250.0, 50.0),
 (250.0, 150.0),
 (250.0, 250.0),
 (250.0, 350.0)]

In [414]:
def select_minimum_penalty_element_grid(img_cds, agent_cd):
    """
    img_cds - объект класса Image_circle_diagrams
    agent_cd - объект класса Agent_circle_diagram
    """
    center_points = []
    penalties = []
    
    for i in range(len(img_cds.center_points)):
        center_point = img_cds.center_points[i]
        img_cd = img_cds.circle_diagrams[i]
        
        cur_cd_penalties = []
        
        for j in range(len(agent_cd.sectors)):
            cur_cd_penalties.append(calculate_penalty_for_diagrams(img_cd.sectors, agent_cd.sectors))
            agent_cd.rotate()
        penalties.append(min(cur_cd_penalties))
        center_points.append(center_point)
    
    minimum_penalty_index = np.argmin(penalties)
    return center_points[minimum_penalty_index], minimum_penalty_index

In [415]:
select_minimum_penalty_element_grid(img_cd, agent_cd)

((150.0, 350.0), 7)

In [ ]:
# TODO: Теперь соберём всё в отдельные модули и полноценно протестируем их работу